In [1]:
import numpy as np
import pandas as pd
from IPython.display import display
import pickle

from tensorflow.keras.models import load_model, Model

from sklearn.model_selection import GroupKFold
from sklearn.metrics import roc_auc_score

In [2]:
input_dir="../input/efficientnet_output_straight/"
dataset_dir="../input/ranzcr-clip-catheter-line-classification/"

input_df=pd.read_csv("./models/efficientnet_output_normalized.csv")
display(input_df)

,StudyInstanceUID,PatientID,0,1,2,3,4,5,6,7,...,2550,2551,2552,2553,2554,2555,2556,2557,2558,2559
0,1.2.826.0.1.3680043.8.498.26697628953273228189...,ec89415d1,0.169359,0.324016,0.173553,0.095382,0.063119,0.165738,0.181906,0.437814,...,0.302277,0.070963,0.354162,0.284205,0.057408,0.101549,0.200983,0.098618,0.276763,0.473490
1,1.2.826.0.1.3680043.8.498.46302891597398758759...,bf4c6da3c,0.365546,0.175262,0.286057,0.205560,0.388747,0.452293,0.021495,0.460131,...,0.232927,0.223938,0.135535,0.042117,0.169312,0.383767,0.095033,0.151809,0.048868,0.117311
2,1.2.826.0.1.3680043.8.498.23819260719748494858...,3fc1c97e5,0.211337,0.303350,0.184402,0.248883,0.202594,0.301177,0.208046,0.416033,...,0.370746,0.315548,0.384991,0.181793,0.134341,0.147335,0.313144,0.356484,0.112562,0.404958
3,1.2.826.0.1.3680043.8.498.68286643202323212801...,c31019814,0.520994,0.385111,0.367907,0.247489,0.097945,0.448468,0.377076,0.108094,...,0.355882,0.156018,0.316916,0.141020,0.041345,0.297808,0.273713,0.329452,0.203884,0.219169
4,1.2.826.0.1.3680043.8.498.10050203009225938259...,207685cd1,0.383775,0.214603,0.263212,0.429333,0.619226,0.365313,0.264323,0.201577,...,0.302169,0.318546,0.097627,0.337127,0.124034,0.266738,0.595066,0.150074,0.454537,0.071115
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30078,1.2.826.0.1.3680043.8.498.74257566841157531124...,5b5b9ac30,0.152407,0.273989,0.429204,0.095627,0.143194,0.151159,0.190642,0.151711,...,0.417215,0.330231,0.257251,0.193722,0.070763,0.415085,0.388353,0.156808,0.239679,0.161568
30079,1.2.826.0.1.3680043.8.498.46510939987173529969...,7192404d8,0.303968,0.299640,0.232293,0.115276,0.309395,0.338309,0.164800,0.221858,...,0.509008,0.143263,0.214838,0.167794,0.075009,0.163606,0.415589,0.059742,0.148673,0.164074
30080,1.2.826.0.1.3680043.8.498.43173270582850645437...,d4d1b066d,0.316939,0.351076,0.589629,0.291154,0.185748,0.232997,0.113028,0.353113,...,0.542412,0.236196,0.170918,0.283692,0.270579,0.275420,0.285688,0.203561,0.118533,0.222272
30081,1.2.826.0.1.3680043.8.498.95092491950130838685...,01a6602b8,0.239194,0.393650,0.438603,0.363903,0.144577,0.259132,0.366278,0.193645,...,0.387640,0.075916,0.316283,0.119015,0.101108,0.645644,0.305596,0.319096,0.330187,0.278469


In [3]:
model=load_model("./models/autoencoder_splits10/")

In [4]:
def evaluate_autoencoder():
    group_kfold=GroupKFold(n_splits=10)
    for n,(train_idx,test_idx) in enumerate(group_kfold.split(input_df.iloc[:,2:],groups=input_df.iloc[:,2])):
        X_train=input_df.iloc[train_idx,2:]
        X_test=input_df.iloc[test_idx,2:]
        score=model.evaluate(X_test,X_test,verbose=0)
        print(f"fold{n} score: {score}")

### AutoEncoderで次元削減する

In [5]:
layer_name="dense_1"
hidden_layer_model=Model(inputs=model.input,outputs=model.get_layer(layer_name).output)

pred=hidden_layer_model.predict(input_df.iloc[:,2:])
features=pd.concat([input_df.iloc[:,:1],pd.DataFrame(pred)],axis=1)
display(features)

,StudyInstanceUID,0,1,2,3,4,5,6,7,8,...,90,91,92,93,94,95,96,97,98,99
0,1.2.826.0.1.3680043.8.498.26697628953273228189...,0.0,3.523285,2.151111,3.073467,3.363639,3.601960,1.177225,0.0,3.399705,...,0.0,1.328145,1.331692,1.144717,2.307059,2.733480,2.879669,4.131839,3.130108,2.915718
1,1.2.826.0.1.3680043.8.498.46302891597398758759...,0.0,2.752369,2.418850,2.875795,3.922628,5.291559,4.010684,0.0,4.176135,...,0.0,1.365769,3.142598,6.856853,2.623820,4.440138,6.352470,7.112836,8.021735,6.673033
2,1.2.826.0.1.3680043.8.498.23819260719748494858...,0.0,5.127551,4.384488,8.046625,5.567223,6.195307,7.095205,0.0,4.194753,...,0.0,1.316996,5.129144,5.434055,5.875688,4.957835,5.093932,4.814246,1.765945,7.694550
3,1.2.826.0.1.3680043.8.498.68286643202323212801...,0.0,2.767875,1.731207,5.098400,2.484329,3.534876,4.957430,0.0,6.703936,...,0.0,0.875055,3.810669,2.164724,1.225728,1.915580,2.081527,1.964060,2.458131,4.171708
4,1.2.826.0.1.3680043.8.498.10050203009225938259...,0.0,5.906751,2.528070,5.296381,2.448073,2.876235,8.141816,0.0,4.746807,...,0.0,1.980754,6.254673,1.629905,3.840276,2.863812,4.631470,4.077279,5.729913,6.588160
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30078,1.2.826.0.1.3680043.8.498.74257566841157531124...,0.0,2.509894,2.170190,3.057890,5.210413,4.008926,4.459599,0.0,4.928145,...,0.0,2.131990,4.075906,4.149724,2.756427,2.651626,5.739603,5.785378,3.671273,3.416882
30079,1.2.826.0.1.3680043.8.498.46510939987173529969...,0.0,3.314908,2.372607,1.440810,2.099609,3.282451,4.099072,0.0,3.791726,...,0.0,1.977184,6.820914,5.900064,3.299438,3.492918,4.771842,4.394142,3.231114,3.616853
30080,1.2.826.0.1.3680043.8.498.43173270582850645437...,0.0,5.604525,2.085924,4.015664,4.236988,6.777108,4.505717,0.0,5.942400,...,0.0,1.665230,5.557427,4.285787,2.033923,4.248517,4.583074,10.027946,7.137775,5.121286
30081,1.2.826.0.1.3680043.8.498.95092491950130838685...,0.0,5.239021,1.398003,3.633277,4.615515,3.244748,3.063015,0.0,6.512488,...,0.0,1.476729,3.201416,1.321704,2.968237,3.272295,2.854025,0.896478,3.066891,2.736391


In [6]:
dataset_dir="../input/ranzcr-clip-catheter-line-classification/"
models_dir="./models/"

n_folds=4
num_features=100
target_cols=['ETT - Abnormal', 'ETT - Borderline', 'ETT - Normal', 'NGT - Abnormal', 'NGT - Borderline', 'NGT - Incompletely Imaged', 'NGT - Normal', 'CVC - Abnormal', 'CVC - Borderline', 'CVC - Normal', 'Swan Ganz Catheter Present']

In [7]:
train=pd.read_csv(dataset_dir+"train.csv")

group_kfold=GroupKFold(n_splits=n_folds)
folds=train.copy()

for n,(train_idx,val_idx) in enumerate(group_kfold.split(train,groups=train["PatientID"].values)):
    folds.loc[val_idx,"fold"]=n

folds["fold"]=folds["fold"].astype(int)
display(folds)

,StudyInstanceUID,ETT - Abnormal,ETT - Borderline,ETT - Normal,NGT - Abnormal,NGT - Borderline,NGT - Incompletely Imaged,NGT - Normal,CVC - Abnormal,CVC - Borderline,CVC - Normal,Swan Ganz Catheter Present,PatientID,fold
0,1.2.826.0.1.3680043.8.498.26697628953273228189...,0,0,0,0,0,0,1,0,0,0,0,ec89415d1,2
1,1.2.826.0.1.3680043.8.498.46302891597398758759...,0,0,1,0,0,1,0,0,0,1,0,bf4c6da3c,2
2,1.2.826.0.1.3680043.8.498.23819260719748494858...,0,0,0,0,0,0,0,0,1,0,0,3fc1c97e5,0
3,1.2.826.0.1.3680043.8.498.68286643202323212801...,0,0,0,0,0,0,0,1,0,0,0,c31019814,0
4,1.2.826.0.1.3680043.8.498.10050203009225938259...,0,0,0,0,0,0,0,0,0,1,0,207685cd1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30078,1.2.826.0.1.3680043.8.498.74257566841157531124...,0,0,1,0,0,0,0,0,1,1,0,5b5b9ac30,2
30079,1.2.826.0.1.3680043.8.498.46510939987173529969...,0,0,0,0,0,0,0,0,0,1,0,7192404d8,2
30080,1.2.826.0.1.3680043.8.498.43173270582850645437...,0,0,1,0,0,1,0,1,0,1,0,d4d1b066d,3
30081,1.2.826.0.1.3680043.8.498.95092491950130838685...,0,0,0,0,0,0,0,0,1,0,0,01a6602b8,1


In [8]:
dataset=pd.merge(folds,features,on="StudyInstanceUID")

fold=0

dataset_train=dataset[dataset["fold"]!=fold]
dataset_test=dataset[dataset["fold"]==fold]
X_train=dataset_train.iloc[:,-num_features:]
X_test=dataset_test.iloc[:,-num_features:]

## パラメータを最適化する

In [10]:
%%time

from optuna.integration import lightgbm as lgb

def optimize_params():
    for i in range(1,12):
        y_train=dataset_train.iloc[:,i]
        y_test=dataset_test.iloc[:,i]

        lgb_train=lgb.Dataset(X_train,label=y_train)
        lgb_test=lgb.Dataset(X_test,label=y_test,reference=lgb_train)
        
        params={
            "task":"train",
            "boosting_type":"gbdt",
            "objective":"binary",
            "metric":"auc",
            "learning_rate":0.01,
            "num_iterations":1000
            # "early_stopping_rounds":200, #early_stopping_roundsを指定しないとbest_iterationは保存されない
        }

        opt=lgb.train(params,lgb_train,valid_sets=lgb_test, verbose_eval=100)
        pickle.dump(opt.params,open(f"{models_dir}autoencoder_smallLR/lgb_params_{i}.pickle","wb"))

optimize_params()

 further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with posi

### 最適化に要した時間
lr num_it, default: 19min 49s  
lr \*0.1, num_it \*10: 2h 16min 1s

## 得られたパラメータで予測する

In [12]:
import lightgbm


def get_pred(train,test,col_name:str):
    X_train=train.iloc[:,-num_features:]
    X_test=test.iloc[:,-num_features:]
    y_train=train[col_name]
    y_test=test[col_name]

    col_index=test.columns.get_loc(col_name)

    lgb_train=lightgbm.Dataset(X_train,label=y_train)
    lgb_test=lightgbm.Dataset(X_test,label=y_test,reference=lgb_train)

    params=pickle.load(open(f"{models_dir}autoencoder_smallLR/lgb_params_{col_index}.pickle","rb"))
    params["early_stopping_rounds"]=500
    params["verbose"]=-1

    model=lightgbm.train(params,lgb_train,valid_sets=lgb_test,verbose_eval=500)
    pred=model.predict(X_test)
    auc=roc_auc_score(y_test,pred)

    return pred,auc


results=pd.DataFrame(columns=target_cols)

for n in range(n_folds):
    print(f"fold - {n}")
    train_n=dataset[dataset["fold"]!=n]
    test_n=dataset[dataset["fold"]==n]


    for col_name in target_cols:
        pred,auc=get_pred(train=train_n,test=test_n,col_name=col_name)
        
        results.loc[f"fold - {n}",col_name]=auc


display(results)
display(results.mean(axis=1),results.mean(axis=1).mean())

Training until validation scores don't improve for 500 rounds
[500]	valid_0's auc: 0.827106
[1000]	valid_0's auc: 0.838635
Did not meet early stopping. Best iteration is:
[690]	valid_0's auc: 0.844788
Training until validation scores don't improve for 500 rounds
[500]	valid_0's auc: 0.814983
[1000]	valid_0's auc: 0.818918
Did not meet early stopping. Best iteration is:
[994]	valid_0's auc: 0.819073
Training until validation scores don't improve for 500 rounds
[500]	valid_0's auc: 0.876277
[1000]	valid_0's auc: 0.881725
Did not meet early stopping. Best iteration is:
[983]	valid_0's auc: 0.881853
Training until validation scores don't improve for 500 rounds
[500]	valid_0's auc: 0.768891
[1000]	valid_0's auc: 0.760288
Did not meet early stopping. Best iteration is:
[512]	valid_0's auc: 0.770872
Training until validation scores don't improve for 500 rounds
[500]	valid_0's auc: 0.74526
[1000]	valid_0's auc: 0.756089
Did not meet early stopping. Best iteration is:
[932]	valid_0's auc: 0.756

,ETT - Abnormal,ETT - Borderline,ETT - Normal,NGT - Abnormal,NGT - Borderline,NGT - Incompletely Imaged,NGT - Normal,CVC - Abnormal,CVC - Borderline,CVC - Normal,Swan Ganz Catheter Present
fold - 0,0.844788,0.819073,0.881853,0.770872,0.756435,0.840699,0.870862,0.615559,0.594236,0.559020,0.850126
fold - 1,0.798686,0.850860,0.869823,0.728818,0.755799,0.836410,0.872900,0.615081,0.571264,0.561733,0.852719
fold - 2,0.718757,0.815182,0.881718,0.774092,0.754020,0.856112,0.854072,0.609029,0.586206,0.552611,0.853255
fold - 3,0.824597,0.836659,0.899588,0.685587,0.752137,0.857212,0.872629,0.606223,0.597855,0.563465,0.858842


fold - 0    0.763957
fold - 1    0.755827
fold - 2    0.750459
fold - 3    0.759527
dtype: float64

0.7574424111251199

## モデルを保存する

In [9]:
import lightgbm


for col_name in target_cols:
    y_train=dataset_train.loc[:,col_name]
    y_test=dataset_test.loc[:,col_name]

    col_index=dataset_train.columns.get_loc(col_name)

    lgb_train=lightgbm.Dataset(X_train,label=y_train)
    lgb_test=lightgbm.Dataset(X_test,label=y_test,reference=lgb_train)
    
    params=pickle.load(open(f"{models_dir}autoencoder/lgb_params_{col_index}.pickle","rb"))    
    params["early_stopping_rounds"]=100
    params["verbose"]=-1

    model=lightgbm.train(params,lgb_train,valid_sets=lgb_test,verbose_eval=200)
    pickle.dump(model,open(f"{models_dir}autoencoder/lgb_model_{col_index}.pickle","wb"))

Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[90]	valid_0's auc: 0.827388
Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[100]	valid_0's auc: 0.818516
Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[97]	valid_0's auc: 0.875658
Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[51]	valid_0's auc: 0.739846
Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[93]	valid_0's auc: 0.726099
Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[100]	valid_0's auc: 0.832617
Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[100]	valid_0's auc: 0.862657
Training until validatio